In [1]:
from Functions.datasets import load_stratified_dataset
import pandas as pd
import nltk
from pycorenlp import StanfordCoreNLP
nlpStanford = StanfordCoreNLP('http://localhost:9000')

In [2]:
df_orig = load_stratified_dataset('Datasets/dataset.csv', 'category' ,1)

In [3]:
df_orig.head()

,organizations,title,locations,text,published,category,site,country,text_length
0,[],Undateable Will Pull Double Duty and Tape Live...,[],Undateable Will Pull Double Duty and Tape Live...,2015-10-24T07:22:00.000+03:00,rawdata\entertainment,oriongazette.com,US,1676
1,[],RentPaidOnline® Releases an API Integration wi...,[],RentPaidOnline® is excited to announce it has ...,2015-10-16T03:00:00.000+03:00,rawdata\financial,prbuzz.com,US,1744
2,[],Govt should honour memory of Charanpreet Dhali...,[],"Fuseworks Media Friday, 23 October, 2015 - 04:...",2015-10-22T18:32:00.000+03:00,rawdata\politics,voxy.co.nz,US,2229
3,[],3 killed when car hits crowd at Okla. State pa...,"['Oklahoma State', 'Okla']",3 killed at Oklahoma State's homecoming parade...,2015-10-25T00:28:00.000+03:00,rawdata\sports,kesq.com,US,4059
4,"['Coral Damage Technology Industry', 'Infosys'...",Fan improves new Star Wars poster by adding mo...,['India'],Noah Consulting being acquired by Infosys in $...,2015-10-20T09:09:00.000+03:00,rawdata\technology,industriesnews.net,US,4372


In [4]:
df = df_orig.copy()

In [5]:
# Drop unused columns
df.drop(['organizations', 'title', 'locations', 'published', 'category', 'site', 'country', 'text_length'], axis=1, inplace=True)

In [6]:
df_tok = df.copy()

In [7]:
pd.set_option('display.max_colwidth', -1)
df.shape

(7, 1)

In [8]:
# Tokenize text
from Functions.function import tokenize
for i in range(df_tok.shape[0]):
    df_tok.text[i] = tokenize(df_tok.text[i])

In [9]:
# Join list of strings to one string
df_tok.text = df_tok.text.str.join(' ')

In [10]:
# Sentiment Analysis untokenized

# Import modules
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob

# Vader
sid = SentimentIntensityAnalyzer()
df[['neg', 'neu', 'pos', 'Vader_Score']] = df['text'].apply(sid.polarity_scores).apply(pd.Series)
df.drop(['neg', 'neu', 'pos'], axis=1, inplace=True)

# TextBlob
df[['TextBlob_Score', 'subjectivity']] = df['text'].apply(lambda x:TextBlob(x).sentiment).apply(pd.Series)
df.drop('subjectivity', axis=1, inplace=True)

In [11]:
# Sentiment Analysis tokenized

# Vader
df_tok[['neg', 'neu', 'pos', 'comp']] = df_tok['text'].apply(sid.polarity_scores).apply(pd.Series)
df_tok.drop(['neg', 'neu', 'pos'], axis=1, inplace=True)

# TextBlob
df_tok[['TextBlob_Score', 'subjectivity']] = df_tok['text'].apply(lambda x:TextBlob(x).sentiment).apply(pd.Series)
df_tok.drop('subjectivity', axis=1, inplace=True)

In [12]:
# Sentiment Analysis with StanfordNLP

#from Functions.function import Sentiment_StanfordNLP
#Stanford_Score = []
#for i in range(df.shape[0]):
#    Stanford_Score.append(Sentiment_StanfordNLP(df.text[i]))
#df['Stanford_Score'] = pd.DataFrame(Stanford_Score)

In [14]:
from Functions.function import Sentiment_StanfordNLP

Sentiment_Score = []
for i in range(df.shape[0]):
    Sentiment_Score.append(Sentiment_StanfordNLP(df.text.values[i]))

df['Stanford_Score'] = pd.DataFrame(Sentiment_Score)